In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [2]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [3]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD

nan

In [5]:
candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an = apply_patterns(df_an)
for cp in candle_patterns:
    df_an[cp] = df_an[cp].astype(float)
df_an.head()

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,body_perc_prev_2,HANGING_MAN,SHOOTING_STAR,SPINNING_TOP,MARUBOZU,ENGULFING,TWEEZER_TOP,TWEEZER_BOTTOM,MORNING_STAR,EVENING_STAR
0,2020-01-03 10:00:00,1.30902,1.31170,1.30856,1.30962,1.30902,1.30962,14.649682,33.757962,19.108280,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-03 11:00:00,1.30963,1.30995,1.30658,1.30667,1.30667,1.30963,2.670623,90.504451,87.833828,...,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2020-01-03 12:00:00,1.30671,1.30872,1.30671,1.30773,1.30671,1.30773,0.000000,50.746269,50.746269,...,19.108280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-03 13:00:00,1.30774,1.30777,1.30525,1.30579,1.30579,1.30774,21.428571,98.809524,77.380952,...,87.833828,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2020-01-03 14:00:00,1.30579,1.30812,1.30530,1.30792,1.30579,1.30792,17.375887,92.907801,75.531915,...,50.746269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_an = BollingerBandsFeature(df_an)
df_an = ATRFeature(df_an)
df_an = KeltnerChannelsFeature(df_an)
df_an = RSIFeature(df_an)
df_an = MACDFeature(df_an)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,Feat_gains,Feat_wins_rma,Feat_losses_rma,Feat_RSI_14,MACD,SIGNAL_MACD,HIST,Feat_MACD,Feat_SIGNAL_MACD,Feat_HIST
0,2020-01-03 10:00:00,1.30902,1.31170,1.30856,1.30962,1.30902,1.30962,14.649682,33.757962,19.108280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-03 11:00:00,1.30963,1.30995,1.30658,1.30667,1.30667,1.30963,2.670623,90.504451,87.833828,...,-0.00295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-03 12:00:00,1.30671,1.30872,1.30671,1.30773,1.30671,1.30773,0.000000,50.746269,50.746269,...,0.00106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-03 13:00:00,1.30774,1.30777,1.30525,1.30579,1.30579,1.30774,21.428571,98.809524,77.380952,...,-0.00194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-03 14:00:00,1.30579,1.30812,1.30530,1.30792,1.30579,1.30792,17.375887,92.907801,75.531915,...,0.00213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,2023-03-24 03:00:00,1.22818,1.22864,1.22613,1.22640,1.22640,1.22818,10.756972,81.673307,70.916335,...,-0.00179,0.000436,0.000624,0.411594,-0.000193,0.000370,-0.000563,-0.000193,0.000370,-0.000563
19995,2023-03-24 04:00:00,1.22640,1.22756,1.22628,1.22703,1.22640,1.22703,9.375000,58.593750,49.218750,...,0.00063,0.000450,0.000579,0.437313,-0.000310,0.000234,-0.000544,-0.000310,0.000234,-0.000544
19996,2023-03-24 05:00:00,1.22703,1.22831,1.22664,1.22825,1.22703,1.22825,23.353293,96.407186,73.053892,...,0.00122,0.000505,0.000538,0.484320,-0.000301,0.000127,-0.000427,-0.000301,0.000127,-0.000427
19997,2023-03-24 06:00:00,1.22825,1.22852,1.22772,1.22850,1.22825,1.22850,66.250000,97.500000,31.250000,...,0.00025,0.000487,0.000499,0.493655,-0.000270,0.000048,-0.000317,-0.000270,0.000048,-0.000317


In [7]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'BB_MA', 'BB_UP',
       'BB_LW', 'Feat_BB_MA_c', 'Feat_BB_UP_c', 'Feat_BB_LW_c', 'Feat_BB_MA_o',
       'Feat_BB_UP_o', 'Feat_BB_LW_o', 'Feat_BB_MA_l', 'Feat_BB_UP_l',
       'Feat_BB_LW_l', 'Feat_BB_MA_h', 'Feat_BB_UP_h', 'Feat_BB_LW_h',
       'Feat_ATR_14', 'EMA', 'KeUp', 'KeLo', 'Feat_EMA_c', 'Feat_KeUp_c',
       'Feat_KeLo_c', 'Feat_EMA_o', 'Feat_KeUp_o', 'Feat_KeLo_o', 'Feat_EMA_h',
       'Feat_KeUp_h', 'Feat_KeLo_h', 'Feat_EMA_l', 'Feat_KeUp_l',
       'Feat_KeLo_l', 'Feat_gains

In [8]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

0.00019

In [9]:
df_an['pivots_l'] = False
df_an['pivots_h'] = False

df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an['pivots_l'] = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an['pivots_h'] = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,HIST,Feat_MACD,Feat_SIGNAL_MACD,Feat_HIST,pivots_l,pivots_h,low_perv,high_perv,low_next,high_next
0,2020-01-06 19:00:00,1.31614,1.31734,1.31553,1.31582,1.31582,1.31614,16.022099,33.701657,17.679558,...,0.000259,0.001814,0.001555,0.000259,False,True,1.31508,1.31687,1.31568,1.31668
1,2020-01-06 20:00:00,1.31583,1.31668,1.31568,1.31570,1.31570,1.31583,2.000000,15.000000,13.000000,...,0.000212,0.001828,0.001616,0.000212,False,False,1.31553,1.31734,1.31544,1.31631
2,2020-01-06 21:00:00,1.31568,1.31631,1.31544,1.31617,1.31568,1.31617,27.586207,83.908046,56.321839,...,0.000185,0.001854,0.001668,0.000185,True,False,1.31568,1.31668,1.31580,1.31664
3,2020-01-06 22:00:00,1.31618,1.31664,1.31580,1.31650,1.31618,1.31650,45.238095,83.333333,38.095238,...,0.000164,0.001877,0.001713,0.000164,False,False,1.31544,1.31631,1.31629,1.31678
4,2020-01-06 23:00:00,1.31650,1.31678,1.31629,1.31673,1.31650,1.31673,42.857143,89.795918,46.938776,...,0.000141,0.001891,0.001751,0.000141,False,False,1.31580,1.31664,1.31610,1.31779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19957,2023-03-24 02:00:00,1.22804,1.22828,1.22672,1.22819,1.22804,1.22819,84.615385,94.230769,9.615385,...,-0.000488,0.000022,0.000510,-0.000488,False,False,1.22791,1.22867,1.22613,1.22864
19958,2023-03-24 03:00:00,1.22818,1.22864,1.22613,1.22640,1.22640,1.22818,10.756972,81.673307,70.916335,...,-0.000563,-0.000193,0.000370,-0.000563,True,True,1.22672,1.22828,1.22628,1.22756
19959,2023-03-24 04:00:00,1.22640,1.22756,1.22628,1.22703,1.22640,1.22703,9.375000,58.593750,49.218750,...,-0.000544,-0.000310,0.000234,-0.000544,False,False,1.22613,1.22864,1.22664,1.22831
19960,2023-03-24 05:00:00,1.22703,1.22831,1.22664,1.22825,1.22703,1.22825,23.353293,96.407186,73.053892,...,-0.000427,-0.000301,0.000127,-0.000427,False,False,1.22628,1.22756,1.22772,1.22852


In [10]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0
SPREAD_IN_POSITON = 0# SPREAD # 0
for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = BUY
                    df_an['SL'][i] = df_an.low[i-1] - SPREAD_IN_POSITON
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] - SPREAD_IN_POSITON) + df_an.close[i]
                    break
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = SELL
                    df_an['SL'][i] = df_an.high[i-1] + SPREAD_IN_POSITON
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] - SPREAD_IN_POSITON) + df_an.close[i]
                    break

C:\Users\meysam\AppData\Local\Temp\ipykernel_8892\942741458.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL[i] = SELL
C:\Users\meysam\AppData\Local\Temp\ipykernel_8892\942741458.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['SL'][i] = df_an.high[i-1] + SPREAD_IN_POSITON
C:\Users\meysam\AppData\Local\Temp\ipykernel_8892\942741458.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['TP'][i] = 2*(df_a

In [11]:
df_an.pivots_l = df_an.pivots_l.astype(float)
df_an.pivots_h = df_an.pivots_h.astype(float)

In [12]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'BB_MA', 'BB_UP',
       'BB_LW', 'Feat_BB_MA_c', 'Feat_BB_UP_c', 'Feat_BB_LW_c', 'Feat_BB_MA_o',
       'Feat_BB_UP_o', 'Feat_BB_LW_o', 'Feat_BB_MA_l', 'Feat_BB_UP_l',
       'Feat_BB_LW_l', 'Feat_BB_MA_h', 'Feat_BB_UP_h', 'Feat_BB_LW_h',
       'Feat_ATR_14', 'EMA', 'KeUp', 'KeLo', 'Feat_EMA_c', 'Feat_KeUp_c',
       'Feat_KeLo_c', 'Feat_EMA_o', 'Feat_KeUp_o', 'Feat_KeLo_o', 'Feat_EMA_h',
       'Feat_KeUp_h', 'Feat_KeLo_h', 'Feat_EMA_l', 'Feat_KeUp_l',
       'Feat_KeLo_l', 'Feat_gains

In [13]:
X = []
Y = [] 
feature_col = [
#     'open', 'high', 'low', 'close',
#                'RSI_14', 'ATR_14',
#     'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
#     'MACD', 'SIGNAL_MACD', 'HIST',
    
    'Feat_BB_MA_c', 'Feat_BB_UP_c', 'Feat_BB_LW_c',
    'Feat_EMA_c', 'Feat_KeUp_c', 'Feat_KeLo_c',
    
    'Feat_BB_MA_o', 'Feat_BB_UP_o', 'Feat_BB_LW_o',
    'Feat_EMA_o', 'Feat_KeUp_o', 'Feat_KeLo_o', 
    
    'Feat_BB_MA_l', 'Feat_BB_UP_l', 'Feat_BB_LW_l',
    'Feat_EMA_l', 'Feat_KeUp_l', 'Feat_KeLo_l',
    
    'Feat_BB_MA_h', 'Feat_BB_UP_h', 'Feat_BB_LW_h',
    'Feat_EMA_h', 'Feat_KeUp_h', 'Feat_KeLo_h',
    
    'Feat_ATR_14',
    'Feat_gains','Feat_wins_rma', 'Feat_losses_rma', 'Feat_RSI_14', 
    'Feat_MACD', 'Feat_SIGNAL_MACD', 'Feat_HIST',
    
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
indeces = []
for i in range(4,len(df_an)):
#     if df_an.SIGNAL[i] != NONE:
    if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
        feature = []
        feature.append(df_an.pivots_h[i-1])
        feature.append(df_an.pivots_l[i-1])
        feature.append(df_an.pivots_h[i-2])
        feature.append(df_an.pivots_l[i-2])
        feature.append(df_an.pivots_h[i-3])
        feature.append(df_an.pivots_l[i-3])
        feature.append(df_an.pivots_h[i-4])
        feature.append(df_an.pivots_l[i-4])
        for fc in feature_col:
            feature.append(df_an[fc][i])
            feature.append(df_an[fc][i-1])
            feature.append(df_an[fc][i-2])
            feature.append(df_an[fc][i-3])
            feature.append(df_an[fc][i-4])

        X.append(feature)
        Y.append(df_an.SIGNAL[i])
        indeces.append(i)
        

In [14]:
print(len(X))
print(len(Y))
print(len(indeces))

8644
8644
8644


In [16]:
print(len(feature))

293


In [15]:
SPLIT_INDEX = int(0.8*len(X))

In [16]:

X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]


In [17]:
print(len(X_train))
print(len(Y_train))
print(len(indeces_train))
print(len(X_test))
print(len(Y_test))
print(len(indeces_test))

6915
6915
6915
1729
1729
1729


In [18]:
# scaler = StandardScaler()  
# scaler.fit(X_train)  
# X_train = scaler.transform(X_train)  
# >>> # apply same transformation to test data
# >>> X_test = scaler.transform(X_test) 


In [19]:
help(MLPClassifier)

Help on class MLPClassifier in module sklearn.neural_network._multilayer_perceptron:

class MLPClassifier(sklearn.base.ClassifierMixin, BaseMultilayerPerceptron)
 |  MLPClassifier(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
 |  
 |  Multi-layer Perceptron classifier.
 |  
 |  This model optimizes the log-loss function using LBFGS or stochastic
 |  gradient descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : tuple, length = n_layers - 2, default=(100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'identity

In [74]:
clf = MLPClassifier( solver='lbfgs', alpha=1e-4,learning_rate_init=1e-4, # adam
#                      tol=0.0000010,
                    hidden_layer_sizes=(200,200), 
                    random_state=1
#                     ,max_iter=100000,
                    verbose=True,warm_start = True,
#                     n_iter_no_change=40
                   )
# clf = MLPClassifier(learning_rate_init=1e-6, warm_start = True, random_state = 2,hidden_layer_sizes= (60,40),max_iter=100000,verbose=True, tol=0.00000010,n_iter_no_change=40)

In [75]:
clf.fit(X_train, Y_train)

KeyboardInterrupt: 

In [23]:
len(clf.coefs_)

3

In [24]:
len(clf.intercepts_)

3

In [44]:
clf.intercepts_[2].shape

(3,)

In [42]:
clf.coefs_[2].shape
 

(200, 3)

In [113]:
x = np.array(X_test[0]).reshape(1,-1)

In [25]:


def softmax(x):
#     return(np.exp(x)/np.exp(x).sum())
    return(np.exp(x - np.max(x)) / np.exp(x - np.max(x)).sum())

def relu(x):
    return (abs(x) + x) / 2

In [26]:
l1 = np.matmul(np.array(X_test),clf.coefs_[0])+clf.intercepts_[0]
l1 = relu(l1)
print(l1.shape)
l2 = np.matmul(l1,clf.coefs_[1])+clf.intercepts_[1]
l2 = relu(l2)
print(l2.shape)
l3 = np.matmul(l2,clf.coefs_[2])+clf.intercepts_[2]
l3.shape

(1729, 200)
(1729, 200)


(1729, 3)

In [27]:
clf.out_activation_

'softmax'

In [28]:
pred = np.array([softmax(x) for x in l3])

In [40]:
pred_lablel = np.argmax(pred, axis=1) 
pred_lablel = pred_lablel-1
pred_lablel

array([ 0, -1,  0, ...,  0,  0, -1], dtype=int64)

In [41]:
(y_perdict==pred_lablel).sum()

1729

In [31]:
y_perdict = clf.predict(X_test)

In [77]:
clf.score(X_train, Y_train)

0.9903109182935648

In [76]:
clf.score(X_test, Y_test)

0.5240023134759977

In [78]:
conf = confusion_matrix(Y_test, y_perdict)
conf

array([[157, 182,   9],
       [210, 638, 201],
       [  6, 182, 144]], dtype=int64)

In [79]:
pickle.dump(clf, open(f"{pair}.pickle", "wb"))

In [80]:
y_prob = clf.predict_proba(X_test)

In [81]:
for i in range(len(Y_test)):
    if Y_test[i] != y_perdict[i]:
        print(Y_test[i])
        print(y_perdict[i])
        print(y_prob[i])

0
-1
[2.67539712e-01 7.32456046e-01 4.24178752e-06]
0
1
[5.07032047e-08 2.68058192e-02 9.73194130e-01]
0
1
[5.84175930e-06 9.99650853e-01 3.43305580e-04]
-1
0
[9.99749848e-01 2.50149538e-04 2.80805215e-09]
0
1
[1.33543662e-08 6.36640025e-02 9.36335984e-01]
1
0
[1.85051773e-07 8.90414724e-01 1.09585091e-01]
0
-1
[9.58541859e-02 9.04113190e-01 3.26239024e-05]
0
1
[3.71637868e-15 9.16318039e-02 9.08368196e-01]
0
-1
[9.93508400e-01 6.49160031e-03 2.74504345e-15]
0
1
[4.46394756e-08 9.99999333e-01 6.22580012e-07]
-1
1
[9.98895601e-01 9.56172122e-04 1.48226477e-04]
-1
0
[3.34980860e-02 9.66501914e-01 5.23841848e-11]
0
-1
[8.46339298e-03 9.91536607e-01 2.45064317e-11]
1
0
[2.46890783e-07 9.87262590e-01 1.27371634e-02]
0
1
[4.36177072e-14 9.34185213e-01 6.58147866e-02]
0
-1
[9.99826896e-01 1.73104196e-04 1.58360498e-13]
0
-1
[0.01057897 0.00234663 0.98707439]
0
-1
[1.75587353e-09 3.42400835e-10 9.99999998e-01]
0
1
[1.84030640e-08 9.84095849e-01 1.59041321e-02]
0
-1
[3.23330880e-03 9.96753536e-

In [82]:
conf[0,0]/(conf[0,:].sum())

0.4511494252873563

In [83]:
conf[1,1]/(conf[1,:].sum())

0.6081982840800763

In [84]:
conf[2,2]/(conf[2,:].sum())

0.43373493975903615

In [85]:
Y_test = np.array(Y_test)

In [86]:
(Y_test==-1).sum()

348

In [87]:
149+186+14

349

In [88]:
indeces_test[0]

15997

In [89]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]-5:].copy()
df_test['SIGNAL'] = 0

for i,idx in enumerate(indeces_test):
    if i > 0:
        if df_test.pivots_h[idx-1] or df_test.pivots_l[idx-1] :
            df_test.SIGNAL.loc[idx] = y_perdict[i]

df_test.reset_index(drop=True, inplace=True)


C:\Users\meysam\AppData\Local\Temp\ipykernel_8892\1575952339.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [90]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51610 entries, 0 to 51609
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    51610 non-null  datetime64[ns]
 1   open    51610 non-null  float64       
 2   high    51610 non-null  float64       
 3   low     51610 non-null  float64       
 4   close   51610 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 2.0 MB


In [91]:
SPREAD = rates.get_spread()
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

0.00011

In [92]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

prepare_data...


In [93]:
df_res_m5 = gt.run_test()

run_test...
Result:
win rate:  0.6239316239316239
 2.0    438
-1.0    264
Name: result, dtype: int64


In [94]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio = 1 #abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


C:\Users\meysam\AppData\Local\Temp\ipykernel_8892\1341523991.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_8892\1341523991.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [95]:
min(df_res_m5['balance'])

79.96698901533449

In [96]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])

In [ ]:
#  cross test
pair_cross = "EURUSD"
NB_H1_CANDLES_CROSS = 2000
rates = Rates(pair_cross, NB_H1_CANDLES_CROSS, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an_cross = rates.get_rates_from_now()
df_an_cross.drop(NB_H1_CANDLES_CROSS-1,inplace=True) 
df_an_cross.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an_cross = apply_patterns(df_an_cross)
for cp in candle_patterns:
    df_an_cross[cp] = df_an_cross[cp].astype(float)

df_an_cross = RSI(df_an_cross)
df_an_cross = BollingerBands(df_an_cross)
df_an_cross = ATR(df_an_cross)
df_an_cross = KeltnerChannels(df_an_cross)
df_an_cross = MACD(df_an_cross)

df_an_cross['pivots_l'] = False
df_an_cross['pivots_h'] = False

df_an_cross['low_perv'] = df_an_cross.low.shift(1)
df_an_cross['high_perv'] = df_an_cross.high.shift(1)
df_an_cross['low_next'] = df_an_cross.low.shift(-1)
df_an_cross['high_next'] = df_an_cross.high.shift(-1)
df_an_cross['pivots_l'] = (df_an_cross.low_perv > df_an_cross.low) & (df_an_cross.low_next > df_an_cross.low)
df_an_cross['pivots_h'] = (df_an_cross.high_perv < df_an_cross.high) & (df_an_cross.high_next < df_an_cross.high)
df_an_cross.dropna(inplace=True)
df_an_cross.reset_index(drop=True, inplace=True)

df_an_cross.head()

0.00011

In [ ]:
BUY = 1
SELL = -1
NONE = 0

df_an_cross['SIGNAL'] = 0
df_an_cross['TP'] = 0
df_an_cross['SL'] = 0

for i in range(1,len(df_an_cross)):
    if df_an_cross.pivots_l[i-1]:
        for j in range(i,len(df_an_cross)):
            if( df_an_cross.low[j] <= df_an_cross.low[i-1] ):
                break
            else:
                if(df_an_cross.high[j] > 2*(df_an_cross.close[i] - df_an_cross.low[i-1] ) + df_an_cross.close[i]):
#                     df_an_cross.SIGNAL[i] = BUY
                    df_an_cross['SL'][i] = df_an_cross.low[i-1] #- SPREAD
                    df_an_cross['TP'][i] = 2*(df_an_cross.close[i] - df_an_cross.low[i-1] ) + df_an_cross.close[i] #+ SPREAD
                    break
    if df_an_cross.pivots_h[i-1]:
        for j in range(i,len(df_an_cross)):
            if( df_an_cross.high[j] >= df_an_cross.high[i-1] ):
                break
            else:
                if(df_an_cross.low[j] < 2*(df_an_cross.close[i] - df_an_cross.high[i-1] ) + df_an_cross.close[i]):
#                     df_an_cross.SIGNAL[i] = SELL
                    df_an_cross['SL'][i] = df_an_cross.high[i-1] #+ SPREAD
                    df_an_cross['TP'][i] = 2*(df_an_cross.close[i] - df_an_cross.high[i-1] ) + df_an_cross.close[i]
                    break

In [ ]:
df_an_cross.pivots_l = df_an_cross.pivots_l.astype(float)
df_an_cross.pivots_h = df_an_cross.pivots_h.astype(float)

In [ ]:
X_cross = []
Y_cross = [] 
feature_col = [
    'open', 'high', 'low', 'close',
               'RSI_14', 'ATR_14',
    'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
    'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
indeces_cross = []
for i in range(4,len(df_an_cross)):
    if df_an_cross.pivots_h[i-1] or df_an_cross.pivots_l[i-1]:
#         print("im here")
        feature = []
        feature.append(df_an_cross.pivots_h[i-1])
        feature.append(df_an_cross.pivots_l[i-1])
        for fc in feature_col:
            feature.append(df_an_cross[fc][i])
            feature.append(df_an_cross[fc][i-1])
            feature.append(df_an_cross[fc][i-2])
            feature.append(df_an_cross[fc][i-3])
            feature.append(df_an_cross[fc][i-4])

        X_cross.append(feature)
        Y_cross.append(df_an_cross.SIGNAL[i])
        indeces_cross.append(i)

In [ ]:
len(feature)

In [ ]:
# clf = pickle.load(open("GBPUSD.pickle", "rb"))

In [ ]:
y_perdict_cross = clf.predict(X_cross)

In [ ]:
confusion_matrix(Y_cross, y_perdict_cross)

In [ ]:
(Y_cross==y_perdict_cross).sum()/len(y_perdict_cross)

In [ ]:
df_test = df_an_cross[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']]#.iloc[indeces_cross[0]-5:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_cross):
    if i > 0:
        if df_test.pivots_l[idx-1] and y_perdict_cross[i] == BUY:
            df_test.SIGNAL.loc[idx] = y_perdict_cross[i]
        if df_test.pivots_h[idx-1] and y_perdict_cross[i] == SELL:
#         if df_test.pivots_h[idx-1] or df_test.pivots_l[idx-1] :
            df_test.SIGNAL.loc[idx] = y_perdict_cross[i]

df_test.reset_index(drop=True, inplace=True)


In [ ]:
rates = Rates(pair_cross, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

In [ ]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

0.0001100000000000545

In [ ]:
df_res_m5 = gt.run_test()

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:200]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])